<a href="https://colab.research.google.com/github/tayfununal/Uniform-Autoencoder-with-Latent-Flow-Matching/blob/main/UAE-with-LFM/LFM_training/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tayfununal/Uniform-Autoencoder-with-Latent-Flow-Matching.git

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torch import Tensor

from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd())

%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/models/mnist_model.ipynb
%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/datasets/mnist_dataset.ipynb

# Flow Matching Model
%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/UAE-with-LFM/models/mnist_flow_model.ipynb

plt.rcParams['font.size'] = 14
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

/content


In [ ]:
# Hyper-Parameters & Settings
batch_size = 500

In [ ]:
# Dataset
train_dataset = MNISTDataset(mode='train')
val_dataset = MNISTDataset(mode='val')

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)

In [ ]:
# Load the UAE model

path = '/content/Uniform-Autoencoder-with-Latent-Flow-Matching/results/mnist/UAE_MNIST'
model = torch.load(path + '.model', weights_only=False)
model.eval()

To_Uniform(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=2000, bias=True)
    (1): SiLU()
    (2): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=2000, out_features=2000, bias=True)
    (4): SiLU()
    (5): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=2000, out_features=2000, bias=True)
    (7): SiLU()
    (8): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=2000, out_features=2000, bias=True)
    (10): SiLU()
    (11): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=2000, out_features=3, bias=True)
    (13): Sigmoid()
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=2000, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2000, out_features=2000, bias=True)
    (3): SiLU()
    (4): L

In [ ]:
# Create the "results" folder
os.makedirs("/content/Uniform-Autoencoder-with-Latent-Flow-Matching/UAE-with-LFM/results/mnist", exist_ok=True)

flow_name = '/content/Uniform-Autoencoder-with-Latent-Flow-Matching/UAE-with-LFM/results/mnist/UAE_Latent_FM'
flow = Flow()

optimizer = torch.optim.Adam(flow.parameters(), 0.0003)
loss_fn = nn.MSELoss()

In [ ]:
train_cost = []
val_cost = []
best_val_loss = float('inf')

for epoch in range(1000):
    epoch_losses = []

    flow.train()
    for x, _ in train_loader:

        with torch.no_grad():
          x_1 = model.encoder(x)  # Encoded representation

        x_0 = torch.rand_like(x_1)
        t = torch.rand(len(x_1), 1)

        x_t = (1 - t) * x_0 + t * x_1
        dx_t = x_1 - x_0

        optimizer.zero_grad()
        pred = flow(t=t, x_t=x_t)
        loss = loss_fn(pred, dx_t)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    mean_train_loss = sum(epoch_losses) / len(epoch_losses)
    train_cost.append(mean_train_loss)

    # Validation
    flow.eval()
    val_losses = []
    with torch.no_grad():
        for x_val, _ in val_loader:
            x_1_val = model.encoder(x_val) # Encoded representation

            x_0_val = torch.rand_like(x_1_val)
            t_val = torch.rand(len(x_1_val), 1)

            x_t_val = (1 - t_val) * x_0_val + t_val * x_1_val
            dx_t_val = x_1_val - x_0_val

            pred_val = flow(t=t_val, x_t=x_t_val)
            val_loss = loss_fn(pred_val, dx_t_val)
            val_losses.append(val_loss.item())

    mean_val_loss = sum(val_losses) / len(val_losses)
    val_cost.append(mean_val_loss)

    # Save the best model based on validation loss
    if mean_val_loss < best_val_loss:
        print('Saved!')
        best_val_loss = mean_val_loss
        torch.save(flow, flow_name + '.model')

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/1000 - Train Loss: {mean_train_loss:.4f} - Val Loss: {mean_val_loss:.4f}")

# Optionally, save the final epoch model separately
torch.save(flow, flow_name + '_final.model')

In [ ]:
# Save to CSV file
train_losses = train_cost
val_losses = val_cost

np.savetxt("/content/Uniform-Autoencoder-with-Latent-Flow-Matching/UAE-with-LFM/results/mnist/losses.csv",
           np.column_stack((train_losses, val_losses)),
           delimiter=",",
           header="train_loss,val_loss",
           comments="")